In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
#Aspect ratio 2
Lx, Ly = (26., 26.)
nx, ny = (300, 300)

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Fourier('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

In [ ]:

def grow_fun(A,B):
    F = (A.data)*(1-B.data)
    F[A.data < 1e-3] = 0
    return F


def grow_operator(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun,
        args = (field1,field2,)
    )

de.operators.parseables['grow'] = grow_operator

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
K =  params['K'].flat[0]
g = params['g'].flat[0]

problem = de.IVP(domain, variables=['rho1','rho2','rhom'])
problem.parameters['D1'] = Diff
problem.parameters['D2'] = 0.25*Diff
problem.parameters['K'] = K
problem.parameters['g0'] = g
problem.parameters['g1'] = g


problem.substitutions['rho_b'] = "rho1+rho2+rhom" 
problem.substitutions["Lap(A)"] = "dx(dx(A)) + dy(dy(A))"

problem.substitutions["grow_nl(A,B)"] = "-A*B"
problem.substitutions["grow_l(A)"] = "-A"    

problem.add_equation("dt(rho1)  - D1*Lap(rho1)  = g0*grow(rho1,rho_b) - K*rho1*(rho2)")
problem.add_equation("dt(rho2)  - D2*Lap(rho2)  = g1*grow(rho2,rho_b) - K*rho2*(rho1)")

problem.add_equation("dt(rhom)  = 2*K*(rho1*rho2)")

In [ ]:
ts = de.timesteppers.RK443
solver =  problem.build_solver(ts)

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']

def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob_np(x,y,x0,y0,r0,w):
    return 0.5*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

def blob(x,y,x0,y0,r0,w):
    return blob_np(x,y,x0,y0,r0,w) + blob_np(x,y,x0+Lx,y0,r0,w) + blob_np(x,y,x0,y0+Ly,r0,w) + blob_np(x,y,x0+Lx,y0+Ly,r0,w)


# Triangular lattice points at: 
# (0.5,sqrt(3)/6) + (n,m*sqrt(3))
# (0.5,5*sqrt(3)/6) + (n,m*sqrt(3))
# (1.,sqrt(3)/6 -sqrt(3)/2) + (n,m*sqrt(3))
# (1.,5*sqrt(3)/6-sqrt(3)/2) + (n,m*sqrt(3))

r0 = 0.6
w = 0.15
s = 0.3
rho1['g'] = blob(x,y,Lx/2 -1/s ,Ly/2,r0,w) + blob(x,y,Lx/2 + 0.5/s,Ly/2+np.sqrt(3)/2/s,r0,w) + blob(x,y,Lx/2 + 0.5/s,Ly/2-np.sqrt(3)/2/s,r0,w)
rho2['g'] = blob(x,y,Lx/2 +1/s,Ly/2,r0,w) + blob(x,y,Lx/2 - 0.5/s,Ly/2+np.sqrt(3)/2/s,r0,w) + blob(x,y,Lx/2 - 0.5/s,Ly/2-np.sqrt(3)/2/s,r0,w)

In [ ]:
solver.stop_sim_time = 200
dt = 0.4*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('square', sim_dt=2., max_writes=500)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rhom')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))


In [ ]:
from dedalus.tools import post
post.merge_process_files("square", cleanup=True)